In [ ]:
import os
os.environ["CUDA_VISIBLE_DEVICES"]="0"
from utils import test_trainer
#import importlib

print('This is a debug file')

Pre-training takes 8.987453699111938s seconds
SVD takes 0.1389787197113037s seconds
EDC Matrix calculation takes 0.0186460018157959s seconds
MADC Matrix calculation takes 0.018960952758789062s seconds
Clustering takes 0.011929512023925781s seconds
Clustering Results: Counter({0: 32, 1: 15, 2: 13})
---------- Round 0 ----------
Round 0, Group 0 has 10 client.
Round 0, Group 1 has 7 client.
Round 0, Group 2 has 3 client.
groupbase.py Test NKS: [108, 121, 60]
Round 0, Test ACC: 0.7163,            Test Loss: 1.2564
Round 0, Group: 0, Test ACC: 0.4537,                    Test Loss: 2.1989
Round 0, Group: 2, Test ACC: 0.8678,                    Test Loss: 0.697
Round 0, Group: 1, Test ACC: 0.8833,                    Test Loss: 0.6882
groupbase.py Auxiliary Model Test NKS: [7371]
Round 0, Auxiliary Model Test ACC: 0.2877,            Auxiliary Model Test Loss: 2.4265
groupbase.py Train NKS: [189.0, 103.0, 200.0]
Round 0, Train ACC: 1.0,            Train Loss: 0.2735
Round 0, Group: 0, Train AC

Round 8, Group 0 has 7 client.
Round 8, Group 1 has 5 client.
Round 8, Group 2 has 8 client.
groupbase.py Test NKS: [568, 399, 309]
Round 8, Test ACC: 0.9067,            Test Loss: 0.5608
Round 8, Group: 0, Test ACC: 0.8732,                    Test Loss: 0.7146
Round 8, Group: 2, Test ACC: 0.9373,                    Test Loss: 0.4218
Round 8, Group: 1, Test ACC: 0.9288,                    Test Loss: 0.4576
groupbase.py Auxiliary Model Test NKS: [7371]
Round 8, Auxiliary Model Test ACC: 0.5945,            Auxiliary Model Test Loss: 1.5073
groupbase.py Train NKS: [814.0, 261.0, 336.0]
Round 8, Train ACC: 1.0,            Train Loss: 0.2279
Round 8, Group: 0, Train ACC: 1.0,                    Train Loss: 0.2237
Round 8, Group: 2, Train ACC: 1.0,                    Train Loss: 0.2374
Round 8, Group: 1, Train ACC: 1.0,                    Train Loss: 0.2308
Round: 8, Training time: 8.507, Test time: 0.595,                 Inter-Group Aggregate time: 0.009
---------- Round 9 ----------
Round 

Round 16, Group 0 has 10 client.
Round 16, Group 1 has 5 client.
Round 16, Group 2 has 5 client.
groupbase.py Test NKS: [1137, 728, 592]
Round 16, Test ACC: 0.9092,            Test Loss: 0.4833
Round 16, Group: 0, Test ACC: 0.8654,                    Test Loss: 0.6005
Round 16, Group: 2, Test ACC: 0.9368,                    Test Loss: 0.4217
Round 16, Group: 1, Test ACC: 0.9595,                    Test Loss: 0.3339
groupbase.py Auxiliary Model Test NKS: [7371]
Round 16, Auxiliary Model Test ACC: 0.6568,            Auxiliary Model Test Loss: 1.3371
groupbase.py Train NKS: [306.0, 299.0, 126.0]
Round 16, Train ACC: 1.0,            Train Loss: 0.1975
Round 16, Group: 0, Train ACC: 1.0,                    Train Loss: 0.1943
Round 16, Group: 2, Train ACC: 1.0,                    Train Loss: 0.1994
Round 16, Group: 1, Train ACC: 1.0,                    Train Loss: 0.2008
Round: 16, Training time: 5.815, Test time: 0.66,                 Inter-Group Aggregate time: 0.008
---------- Round 17 --

Round 24, Group 0 has 6 client.
Round 24, Group 1 has 5 client.
Round 24, Group 2 has 9 client.
groupbase.py Test NKS: [1485, 937, 695]
Round 24, Test ACC: 0.907,            Test Loss: 0.4829
Round 24, Group: 0, Test ACC: 0.8599,                    Test Loss: 0.6073
Round 24, Group: 2, Test ACC: 0.937,                    Test Loss: 0.4231
Round 24, Group: 1, Test ACC: 0.9669,                    Test Loss: 0.2976
groupbase.py Auxiliary Model Test NKS: [7371]
Round 24, Auxiliary Model Test ACC: 0.6682,            Auxiliary Model Test Loss: 1.2457
groupbase.py Train NKS: [98.0, 337.0, 537.0]
Round 24, Train ACC: 1.0,            Train Loss: 0.1882
Round 24, Group: 0, Train ACC: 1.0,                    Train Loss: 0.1798
Round 24, Group: 2, Train ACC: 1.0,                    Train Loss: 0.1898
Round 24, Group: 1, Train ACC: 1.0,                    Train Loss: 0.1888
Round: 24, Training time: 6.885, Test time: 0.76,                 Inter-Group Aggregate time: 0.01
---------- Round 25 -------

Round 32, Group 0 has 10 client.
Round 32, Group 1 has 4 client.
Round 32, Group 2 has 6 client.
groupbase.py Test NKS: [1708, 1283, 1011]
Round 32, Test ACC: 0.8956,            Test Loss: 0.452
Round 32, Group: 0, Test ACC: 0.8829,                    Test Loss: 0.4871
Round 32, Group: 2, Test ACC: 0.848,                    Test Loss: 0.5748
Round 32, Group: 1, Test ACC: 0.9773,                    Test Loss: 0.2367
groupbase.py Auxiliary Model Test NKS: [7371]
Round 32, Auxiliary Model Test ACC: 0.6338,            Auxiliary Model Test Loss: 1.2441
groupbase.py Train NKS: [956.0, 124.0, 170.0]
Round 32, Train ACC: 1.0,            Train Loss: 0.1545
Round 32, Group: 0, Train ACC: 1.0,                    Train Loss: 0.1515
Round 32, Group: 2, Train ACC: 1.0,                    Train Loss: 0.1596
Round 32, Group: 1, Train ACC: 1.0,                    Train Loss: 0.1676
Round: 32, Training time: 8.775, Test time: 0.808,                 Inter-Group Aggregate time: 0.009
---------- Round 33 -

Round 40, Group 0 has 8 client.
Round 40, Group 1 has 7 client.
Round 40, Group 2 has 5 client.
groupbase.py Test NKS: [2052, 1381, 1144]
Round 40, Test ACC: 0.9159,            Test Loss: 0.4137
Round 40, Group: 0, Test ACC: 0.8519,                    Test Loss: 0.6203
Round 40, Group: 2, Test ACC: 0.9623,                    Test Loss: 0.2634
Round 40, Group: 1, Test ACC: 0.9747,                    Test Loss: 0.2247
groupbase.py Auxiliary Model Test NKS: [7371]
Round 40, Auxiliary Model Test ACC: 0.563,            Auxiliary Model Test Loss: 1.3258
groupbase.py Train NKS: [780.0, 142.0, 1367.0]
Round 40, Train ACC: 1.0,            Train Loss: 0.1425
Round 40, Group: 0, Train ACC: 1.0,                    Train Loss: 0.1345
Round 40, Group: 2, Train ACC: 1.0,                    Train Loss: 0.1508
Round 40, Group: 1, Train ACC: 1.0,                    Train Loss: 0.1462
Round: 40, Training time: 15.188, Test time: 0.833,                 Inter-Group Aggregate time: 0.009
---------- Round 41

Round 48, Group 0 has 12 client.
Round 48, Group 1 has 4 client.
Round 48, Group 2 has 4 client.
groupbase.py Test NKS: [2373, 1439, 1221]
Round 48, Test ACC: 0.9523,            Test Loss: 0.2956
Round 48, Group: 0, Test ACC: 0.9317,                    Test Loss: 0.3618
Round 48, Group: 2, Test ACC: 0.9646,                    Test Loss: 0.2618
Round 48, Group: 1, Test ACC: 0.9779,                    Test Loss: 0.2069
groupbase.py Auxiliary Model Test NKS: [7371]
Round 48, Auxiliary Model Test ACC: 0.6365,            Auxiliary Model Test Loss: 1.1745
groupbase.py Train NKS: [1954.0, 52.0, 722.0]
Round 48, Train ACC: 1.0,            Train Loss: 0.1262
Round 48, Group: 0, Train ACC: 1.0,                    Train Loss: 0.1215
Round 48, Group: 2, Train ACC: 1.0,                    Train Loss: 0.1445
Round 48, Group: 1, Train ACC: 1.0,                    Train Loss: 0.1378
Round: 48, Training time: 15.176, Test time: 0.858,                 Inter-Group Aggregate time: 0.008
---------- Round 4

Round 56, Group 0 has 13 client.
Round 56, Group 1 has 2 client.
Round 56, Group 2 has 5 client.
groupbase.py Test NKS: [2638, 1484, 1278]
Round 56, Test ACC: 0.957,            Test Loss: 0.2692
Round 56, Group: 0, Test ACC: 0.9393,                    Test Loss: 0.3295
Round 56, Group: 2, Test ACC: 0.971,                    Test Loss: 0.2295
Round 56, Group: 1, Test ACC: 0.9773,                    Test Loss: 0.1905
groupbase.py Auxiliary Model Test NKS: [7371]
Round 56, Auxiliary Model Test ACC: 0.6602,            Auxiliary Model Test Loss: 1.1261
groupbase.py Train NKS: [822.0, 178.0, 50.0]
Round 56, Train ACC: 1.0,            Train Loss: 0.1147
Round 56, Group: 0, Train ACC: 1.0,                    Train Loss: 0.1098
Round 56, Group: 2, Train ACC: 1.0,                    Train Loss: 0.135
Round 56, Group: 1, Train ACC: 1.0,                    Train Loss: 0.1235
Round: 56, Training time: 7.798, Test time: 0.954,                 Inter-Group Aggregate time: 0.01
---------- Round 57 ----

Round 64, Group 0 has 7 client.
Round 64, Group 1 has 6 client.
Round 64, Group 2 has 7 client.
groupbase.py Test NKS: [2857, 1543, 1372]
Round 64, Test ACC: 0.8479,            Test Loss: 0.552
Round 64, Group: 0, Test ACC: 0.7249,                    Test Loss: 0.8959
Round 64, Group: 2, Test ACC: 0.9657,                    Test Loss: 0.2372
Round 64, Group: 1, Test ACC: 0.9716,                    Test Loss: 0.1901
groupbase.py Auxiliary Model Test NKS: [7371]
Round 64, Auxiliary Model Test ACC: 0.5915,            Auxiliary Model Test Loss: 1.2788
groupbase.py Train NKS: [775.0, 252.0, 157.0]
Round 64, Train ACC: 1.0,            Train Loss: 0.1085
Round 64, Group: 0, Train ACC: 1.0,                    Train Loss: 0.1004
Round 64, Group: 2, Train ACC: 1.0,                    Train Loss: 0.128
Round 64, Group: 1, Train ACC: 1.0,                    Train Loss: 0.1173
Round: 64, Training time: 8.53, Test time: 0.932,                 Inter-Group Aggregate time: 0.009
---------- Round 65 ---

Round 72, Group 0 has 8 client.
Round 72, Group 1 has 6 client.
Round 72, Group 2 has 6 client.
groupbase.py Test NKS: [2982, 1605, 1412]
Round 72, Test ACC: 0.9352,            Test Loss: 0.298
Round 72, Group: 0, Test ACC: 0.9239,                    Test Loss: 0.3327
Round 72, Group: 2, Test ACC: 0.9184,                    Test Loss: 0.3421
Round 72, Group: 1, Test ACC: 0.978,                    Test Loss: 0.1748
groupbase.py Auxiliary Model Test NKS: [7371]
Round 72, Auxiliary Model Test ACC: 0.6223,            Auxiliary Model Test Loss: 1.1672
groupbase.py Train NKS: [152.0, 376.0, 106.0]
Round 72, Train ACC: 1.0,            Train Loss: 0.1044
Round 72, Group: 0, Train ACC: 1.0,                    Train Loss: 0.1001
Round 72, Group: 2, Train ACC: 1.0,                    Train Loss: 0.1039
Round 72, Group: 1, Train ACC: 1.0,                    Train Loss: 0.1125
Round: 72, Training time: 5.226, Test time: 0.907,                 Inter-Group Aggregate time: 0.007
---------- Round 73 --

In [ ]:
import tensorflow as tf
from tensorflow.keras.layers import Dense, Input, Reshape, Conv2D, MaxPool2D, Flatten
from tensorflow.keras import Sequential

model = Sequential()
# Input Layer
model.add(Input(shape=(784,)))
# Reshape Layer
model.add(Reshape((28, 28, 1)))
# Conv Layer
model.add(Conv2D(24, 5, padding='same', activation='relu', input_shape=(28, 28, 1)))
# MaxPool Layer
model.add(MaxPool2D(pool_size=(2, 2), strides=(2, 2)))
# Flatten Layer
model.add(Flatten())
# Dense Layer
model.add(Dense(256, 'relu'))
# Output Layer
model.add(Dense(10, 'softmax'))

#opt = tf.keras.optimizers.SGD(learning_rate=lr)
opt = tf.keras.optimizers.Adam(learning_rate=0.001)
loss_fn = tf.keras.losses.SparseCategoricalCrossentropy()
model.compile(opt, loss_fn, metrics=['accuracy'])

In [ ]:
import numpy as np
# Model / data parameters
num_classes = 10
input_shape = (28, 28, 1)

# the data, split between train and test sets
(x_train, y_train), (x_test, y_test) = tf.keras.datasets.mnist.load_data()

# Scale images to the [0, 1] range
x_train = x_train.astype("float32") / 255
x_test = x_test.astype("float32") / 255
# Make sure images have shape (28, 28, 1)
x_train = np.expand_dims(x_train, -1)
x_test = np.expand_dims(x_test, -1)
print("x_train shape:", x_train.shape)
print(x_train.shape[0], "train samples")
print(x_test.shape[0], "test samples")


# convert class vectors to binary class matrices
y_train = tf.keras.utils.to_categorical(y_train, num_classes)
y_test = tf.keras.utils.to_categorical(y_test, num_classes)

In [ ]:
model = tf.keras.Sequential(
    [
        tf.keras.Input(shape=input_shape),
        tf.keras.layers.Conv2D(32, kernel_size=(3, 3), activation="relu"),
        tf.keras.layers.MaxPooling2D(pool_size=(2, 2)),
        tf.keras.layers.Conv2D(64, kernel_size=(3, 3), activation="relu"),
        tf.keras.layers.MaxPooling2D(pool_size=(2, 2)),
        tf.keras.layers.Flatten(),
        tf.keras.layers.Dropout(0.5),
        tf.keras.layers.Dense(num_classes, activation="softmax"),
    ]
)
model.summary()

In [ ]:
batch_size = 128
epochs = 15

model.compile(loss="categorical_crossentropy", optimizer="adam", metrics=["accuracy"])

model.fit(x_train, y_train, batch_size=batch_size, epochs=epochs, validation_split=0.1)

In [ ]:
import input_data
mnist = input_data.read_data_sets("MNIST_data/", one_hot=True)

In [ ]:
import tensorflow as tf
from tensorflow.keras.layers import Dense, Input
from tensorflow.keras import Sequential

model = Sequential()
# Input Layer
model.add(Input(shape=(784,)))
# Output Layer
model.add(Dense(10, 'softmax', kernel_regularizer=tf.keras.regularizers.l2(0.001)))
    
opt = tf.keras.optimizers.SGD(learning_rate=0.003)
loss_fn = tf.keras.losses.SparseCategoricalCrossentropy()
model.compile(opt, loss_fn, metrics=['accuracy'])

In [ ]:
x = model.get_weights()
print(len(x[0]))

In [ ]:
import numpy as np
random_idx = np.arange(5)
print(random_idx[1:2])

In [ ]:
a = {'x':[],'y':[]}
b = {'x':[],'y':[]}
if a is b:
    print('TRUE')
else:
    print('FALSE')